In [ ]:
%pylab inline

In [ ]:
!ls data/

In [ ]:
try:
    import messagepack
except ImportError:
    !pip install msgpack
    import msgpack


In [ ]:
savestream_filename = "data/Album 0/abb977e2-dfba-4a9c-b3d3-c915065160e7/states.savestream"

In [ ]:
import msgpack

# Read and unpack the savestream file
with open(savestream_filename, 'rb') as f:
    unpacker = msgpack.Unpacker(f, raw=False)
    states = list(unpacker)[0]

print(f"Number of states: {len(states)}")

In [ ]:

sizes = [len(msgpack.packb(state)) for state in states]
sizes


In [ ]:
bar(arange(len(sizes)),sizes)
yscale('log')

In [ ]:
import os

# Create thumbnails directory
video_dir = "data/Album 0/abb977e2-dfba-4a9c-b3d3-c915065160e7"
video_file = os.path.join(video_dir, "response.webm")
thumbs_dir = os.path.join(video_dir, "thumbnails")
os.makedirs(thumbs_dir, exist_ok=True)

# Generate thumbnails at 1fps (one per second)
!ffmpeg -i "{video_file}" -vf "fps=1" "{thumbs_dir}/thumb_%04d.png"


In [ ]:
from PIL import Image
import glob

# Subsampling ratio: show thumbnail every k frames
k = 2  # Show every kth thumbnail

# Load all thumbnails
thumb_files = sorted(glob.glob(os.path.join(thumbs_dir, "thumb_*.png")))
thumbs = [Image.open(f) for f in thumb_files]

print(f"Loaded {len(thumbs)} thumbnails, showing every {k}th")

# Create figure with two rows: thumbnails on top, size chart below
fig, (ax1, ax2) = subplots(2, 1, figsize=(20, 8), gridspec_kw={'height_ratios': [3, 1]})

# Top: concatenate subsampled thumbnails horizontally
thumb_height = 100
thumb_arrays = []
for i in range(0, len(thumbs), k):
    thumb = thumbs[i]
    # Scale down maintaining aspect ratio
    aspect = thumb.width / thumb.height
    new_width = int(thumb_height * aspect)
    resized = thumb.resize((new_width, thumb_height), Image.Resampling.LANCZOS)
    thumb_arrays.append(array(resized))

# Concatenate horizontally
filmstrip = concatenate(thumb_arrays, axis=1)
ax1.imshow(filmstrip)
ax1.axis('off')
ax1.set_title(f'Video Thumbnails (every {k} seconds)', fontsize=14)

# Bottom: plot ALL sizes (one bar per frame)
ax2.bar(arange(len(sizes)), sizes, width=1.0, edgecolor='none')
ax2.set_xlabel('State Index')
ax2.set_ylabel('Size (bytes)')
ax2.set_title('Encoded State Sizes (all frames)', fontsize=14)
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

tight_layout()
